# 🎛️ Interactive Controller Demo - Live Parameter Tuning

This notebook transforms our **excellent documentation** into an **interactive research tool**.

Use the sliders below to adjust controller parameters in real-time and see immediate effects on pendulum control performance!

## Features
- 🎯 **Real-time parameter tuning** with immediate visual feedback
- 📊 **Live performance metrics** (settling time, RMS control effort)
- 🎬 **Animated pendulum visualization** showing control behavior
- 📈 **Comparative analysis** between different controller types
- 🔬 **Educational insights** into sliding mode control theory

---

In [ ]:
# Essential imports for interactive control demonstration
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import sys
import os

# Add project root to path for imports
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

# Import our documented modules (showcasing Task 2 documentation)
from src.config import load_config
from src.controllers.factory import create_controller
from src.core.dynamics import DIPDynamics
from src.core.dynamics_full import FullDIPDynamics
from src.core.simulation_runner import run_simulation
from src.utils.visualization import Visualizer
from src.utils.statistics import confidence_interval  # Featured in our enhanced docs!
from src.utils.control_outputs import ClassicalSMCOutput, AdaptiveSMCOutput, STAOutput, HybridSTAOutput

# Configure matplotlib for notebook
%matplotlib widget
plt.style.use('seaborn-v0_8')

print("🚀 Interactive Controller Demo Ready!")
print("📚 Using professionally documented modules from Task 2")

## 📋 Load Configuration

Loading the validated configuration system documented in our enhanced `src.config` module.

In [ ]:
# Load configuration using documented config system
config = load_config('../config.yaml')

# Setup dynamics models (both simple and full)
simple_dynamics = DIPDynamics(config.physics)
full_dynamics = FullDIPDynamics(config.physics)

print("✅ Configuration loaded successfully")
print(f"📊 Available controllers: {list(config.controllers.keys())}")
print(f"⚙️ Physics parameters loaded: {len(config.physics.__dict__)} parameters")

## 🎛️ Interactive Controller Parameters

Adjust controller gains in real-time and observe the effects on system performance!

### Controller Types:
- **Classical SMC**: Traditional sliding mode with boundary layer
- **Super-Twisting (STA)**: Continuous, finite-time convergent control
- **Adaptive SMC**: Online gain adaptation for uncertainty
- **Hybrid Adaptive STA**: Advanced hybrid approach

In [ ]:
# Interactive controller selection and parameter tuning
class InteractiveControllerDemo:
    def __init__(self):
        self.config = config
        self.setup_widgets()
        self.setup_output_areas()
        
    def setup_widgets(self):
        """Create interactive widgets for parameter control"""
        
        # Controller selection
        self.controller_dropdown = widgets.Dropdown(
            options=['classical_smc', 'sta_smc', 'adaptive_smc', 'hybrid_adaptive_sta_smc'],
            value='classical_smc',
            description='Controller:',
            style={'description_width': 'initial'}
        )
        
        # Dynamics model selection
        self.dynamics_checkbox = widgets.Checkbox(
            value=False,
            description='Use Full Nonlinear Dynamics',
            style={'description_width': 'initial'}
        )
        
        # Simulation parameters
        self.duration_slider = widgets.FloatSlider(
            value=5.0, min=1.0, max=20.0, step=0.5,
            description='Duration (s):',
            style={'description_width': 'initial'}
        )
        
        # Initial disturbance
        self.disturbance_slider = widgets.FloatSlider(
            value=0.1, min=0.0, max=0.5, step=0.01,
            description='Initial θ1 (rad):',
            style={'description_width': 'initial'}
        )
        
        # Controller gain sliders (will be updated based on controller type)
        self.gain_sliders = []
        self.update_gain_sliders()
        
        # Control buttons
        self.run_button = widgets.Button(
            description='🚀 Run Simulation',
            button_style='success',
            layout=widgets.Layout(width='200px')
        )
        
        self.auto_update_checkbox = widgets.Checkbox(
            value=False,
            description='Auto-update on parameter change',
            style={'description_width': 'initial'}
        )
        
        # Set up callbacks
        self.controller_dropdown.observe(self.on_controller_change, names='value')
        self.run_button.on_click(self.run_simulation)
        
        # Auto-update setup
        for slider in [self.duration_slider, self.disturbance_slider]:
            slider.observe(self.on_parameter_change, names='value')
            
    def setup_output_areas(self):
        """Create output areas for results"""
        self.metrics_output = widgets.Output()
        self.plot_output = widgets.Output()
        self.animation_output = widgets.Output()
        
    def update_gain_sliders(self):
        """Update gain sliders based on selected controller"""
        controller_type = self.controller_dropdown.value
        default_gains = self.config.controller_defaults[controller_type]['gains']
        
        # Clear existing sliders
        self.gain_sliders = []
        
        # Create new sliders based on controller type
        gain_labels = {
            'classical_smc': ['k1', 'k2', 'k3', 'λ1', 'λ2', 'λ3'],
            'sta_smc': ['k1', 'k2', 'k3', 'α1', 'α2', 'α3'],
            'adaptive_smc': ['k1', 'k2', 'k3', 'γ1', 'γ2'],
            'hybrid_adaptive_sta_smc': ['c1', 'λ1', 'c2', 'λ2']
        }[controller_type]
        
        for i, (label, default_val) in enumerate(zip(gain_labels, default_gains)):
            slider = widgets.FloatSlider(
                value=default_val,
                min=0.1,
                max=20.0,
                step=0.1,
                description=f'{label}:',
                style={'description_width': 'initial'}
            )
            slider.observe(self.on_parameter_change, names='value')
            self.gain_sliders.append(slider)
            
    def on_controller_change(self, change):
        """Handle controller type change"""
        self.update_gain_sliders()
        self.display_interface()
        if self.auto_update_checkbox.value:
            self.run_simulation(None)
            
    def on_parameter_change(self, change):
        """Handle parameter changes"""
        if self.auto_update_checkbox.value:
            self.run_simulation(None)
            
    def run_simulation(self, button):
        """Run simulation with current parameters"""
        with self.plot_output:
            clear_output(wait=True)
            print("🔄 Running simulation...")
            
        try:
            # Get current parameters
            controller_type = self.controller_dropdown.value
            gains = [slider.value for slider in self.gain_sliders]
            duration = self.duration_slider.value
            initial_theta1 = self.disturbance_slider.value
            
            # Setup initial state with disturbance
            initial_state = np.array([0.0, initial_theta1, 0.0, 0.0, 0.0, 0.0])
            
            # Select dynamics model
            dynamics = full_dynamics if self.dynamics_checkbox.value else simple_dynamics
            
            # Create controller
            controller = create_controller(controller_type, config=self.config, gains=gains)
            
            # Run simulation
            t, x, u = run_simulation(
                controller=controller,
                dynamics_model=dynamics,
                sim_time=duration,
                dt=0.02,
                initial_state=initial_state
            )
            
            # Compute and display metrics
            self.display_metrics(t, x, u, controller_type, gains)
            
            # Create plots
            self.display_plots(t, x, u)
            
            # Create animation preview
            self.display_animation_preview(t, x, u, dynamics)
            
        except Exception as e:
            with self.plot_output:
                clear_output(wait=True)
                print(f"❌ Simulation failed: {str(e)}")
                
    def display_metrics(self, t, x, u, controller_type, gains):
        """Display performance metrics using our documented statistics module"""
        with self.metrics_output:
            clear_output(wait=True)
            
            # Calculate settling time
            within_bounds = (np.abs(x[:, 0]) < 0.02) & (np.abs(x[:, 1]) < 0.05) & (np.abs(x[:, 2]) < 0.05)
            settle_idx = len(t) - 1
            for i in range(len(t)):
                if np.all(within_bounds[i:]):
                    settle_idx = i
                    break
            settling_time = t[settle_idx]
            
            # RMS control effort
            rms_control = np.sqrt(np.mean(u**2))
            
            # Peak values
            peak_x = np.max(np.abs(x[:, 0]))
            peak_theta1 = np.rad2deg(np.max(np.abs(x[:, 1])))
            peak_theta2 = np.rad2deg(np.max(np.abs(x[:, 2])))
            max_u = np.max(np.abs(u))
            
            # Display metrics with confidence intervals (showcasing documented function!)
            control_samples = u[abs(u) > 0.1]  # Non-trivial control efforts
            if len(control_samples) > 10:
                mean_control, ci_half = confidence_interval(control_samples, confidence=0.95)
                ci_display = f" (95% CI: ±{ci_half:.2f})"
            else:
                ci_display = ""
                
            print(f"🎯 {controller_type.upper()} Performance Metrics")
            print(f"📊 Gains: {[f'{g:.1f}' for g in gains]}")
            print(f"⏱️  Settling Time: {settling_time:.2f} s")
            print(f"⚡ RMS Control: {rms_control:.2f} N{ci_display}")
            print(f"📏 Peak |x|: {peak_x:.3f} m")
            print(f"📐 Peak |θ1|: {peak_theta1:.1f}°")
            print(f"📐 Peak |θ2|: {peak_theta2:.1f}°")
            print(f"💪 Max |u|: {max_u:.1f} N")
            
    def display_plots(self, t, x, u):
        """Create comprehensive performance plots"""
        with self.plot_output:
            clear_output(wait=True)
            
            fig, axes = plt.subplots(2, 2, figsize=(12, 8))
            fig.suptitle('Real-time Controller Performance Analysis', fontsize=14, fontweight='bold')
            
            # Cart position
            axes[0, 0].plot(t, x[:, 0], 'b-', linewidth=2)
            axes[0, 0].set_ylabel('Cart Position x (m)')
            axes[0, 0].grid(True, alpha=0.3)
            axes[0, 0].set_title('Cart Stabilization')
            
            # Pendulum angles
            axes[0, 1].plot(t, np.rad2deg(x[:, 1]), 'r-', label='θ1', linewidth=2)
            axes[0, 1].plot(t, np.rad2deg(x[:, 2]), 'g-', label='θ2', linewidth=2)
            axes[0, 1].set_ylabel('Angles (degrees)')
            axes[0, 1].legend()
            axes[0, 1].grid(True, alpha=0.3)
            axes[0, 1].set_title('Pendulum Balance')
            
            # Control input
            axes[1, 0].plot(t[:-1], u, 'k-', linewidth=2)
            axes[1, 0].set_ylabel('Control Force u (N)')
            axes[1, 0].set_xlabel('Time (s)')
            axes[1, 0].grid(True, alpha=0.3)
            axes[1, 0].set_title('Control Effort')
            
            # Phase portrait (θ1 vs θ1_dot)
            axes[1, 1].plot(x[:, 1], x[:, 4], 'purple', alpha=0.7, linewidth=2)
            axes[1, 1].scatter(x[0, 1], x[0, 4], color='green', s=100, label='Start', zorder=5)
            axes[1, 1].scatter(x[-1, 1], x[-1, 4], color='red', s=100, label='End', zorder=5)
            axes[1, 1].set_xlabel('θ1 (rad)')
            axes[1, 1].set_ylabel('θ̇1 (rad/s)')
            axes[1, 1].legend()
            axes[1, 1].grid(True, alpha=0.3)
            axes[1, 1].set_title('Phase Portrait')
            
            plt.tight_layout()
            plt.show()
            
    def display_animation_preview(self, t, x, u, dynamics):
        """Show pendulum animation preview"""
        with self.animation_output:
            clear_output(wait=True)
            print("🎬 Pendulum Animation Preview")
            print(f"📈 Total frames: {len(x)}")
            print(f"⏱️ Duration: {t[-1]:.1f} seconds")
            print("💡 Tip: Use the Streamlit app for full interactive animation!")
            
            # Show key frames as static images
            fig, axes = plt.subplots(1, 3, figsize=(12, 4))
            frames_to_show = [0, len(x)//2, -1]
            frame_labels = ['Initial', 'Midpoint', 'Final']
            
            for i, (frame_idx, label) in enumerate(zip(frames_to_show, frame_labels)):
                ax = axes[i]
                state = x[frame_idx]
                self.draw_pendulum_frame(ax, state, dynamics, title=f'{label} (t={t[frame_idx]:.1f}s)')
                
            plt.tight_layout()
            plt.show()
            
    def draw_pendulum_frame(self, ax, state, dynamics, title):
        """Draw a single frame of the pendulum"""
        x_pos, th1, th2 = state[0], state[1], state[2]
        l1, l2 = dynamics.l1, dynamics.l2
        
        # Calculate positions
        cart_x, cart_y = x_pos, 0.0
        p1_x = cart_x + (2 * l1) * np.sin(th1)
        p1_y = cart_y + (2 * l1) * np.cos(th1)
        p2_x = p1_x + (2 * l2) * np.sin(th2)
        p2_y = p1_y + (2 * l2) * np.cos(th2)
        
        # Draw pendulum
        ax.plot([cart_x, p1_x], [cart_y, p1_y], 'ro-', linewidth=3, markersize=8, label='Link 1')
        ax.plot([p1_x, p2_x], [p1_y, p2_y], 'bo-', linewidth=3, markersize=8, label='Link 2')
        
        # Draw cart
        cart_width, cart_height = 0.4, 0.2
        cart = plt.Rectangle((cart_x - cart_width/2, -cart_height/2), 
                           cart_width, cart_height, 
                           facecolor='gray', edgecolor='black')
        ax.add_patch(cart)
        
        # Draw ground
        ax.axhline(y=-cart_height/2, color='black', linewidth=2)
        
        # Set equal aspect and limits
        ax.set_aspect('equal')
        ax.set_xlim(x_pos - 2, x_pos + 2)
        ax.set_ylim(-1, 3)
        ax.grid(True, alpha=0.3)
        ax.set_title(title)
        
        # Add angle annotations
        ax.text(0.02, 0.98, f'θ1 = {np.rad2deg(th1):.1f}°\nθ2 = {np.rad2deg(th2):.1f}°', 
                transform=ax.transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
            
    def display_interface(self):
        """Display the complete interactive interface"""
        # Controller selection section
        controller_box = widgets.VBox([
            widgets.HTML("<h3>🎛️ Controller Configuration</h3>"),
            self.controller_dropdown,
            self.dynamics_checkbox
        ])
        
        # Simulation parameters section  
        sim_params_box = widgets.VBox([
            widgets.HTML("<h3>⚙️ Simulation Parameters</h3>"),
            self.duration_slider,
            self.disturbance_slider
        ])
        
        # Controller gains section
        gains_box = widgets.VBox([
            widgets.HTML("<h3>🎯 Controller Gains</h3>")
        ] + self.gain_sliders)
        
        # Control section
        control_box = widgets.VBox([
            widgets.HTML("<h3>🚀 Simulation Control</h3>"),
            self.auto_update_checkbox,
            self.run_button
        ])
        
        # Layout everything
        interface = widgets.VBox([
            widgets.HBox([controller_box, sim_params_box]),
            gains_box,
            control_box,
            widgets.HTML("<hr><h3>📊 Performance Metrics</h3>"),
            self.metrics_output,
            widgets.HTML("<h3>📈 Analysis Plots</h3>"),
            self.plot_output,
            widgets.HTML("<h3>🎬 Animation Preview</h3>"),
            self.animation_output
        ])
        
        return interface

# Create and display the interactive demo
demo = InteractiveControllerDemo()
interface = demo.display_interface()
display(interface)

print("🎉 Interactive Controller Demo is ready!")
print("💡 Adjust parameters above and click 'Run Simulation' to see real-time results")
print("🔄 Enable 'Auto-update' for immediate feedback on parameter changes")

## 🎓 Educational Insights & Control Theory

### Understanding the Parameters:

**Classical SMC Parameters:**
- `k1, k2, k3`: Sliding surface gains (higher = faster convergence, more aggressive)
- `λ1, λ2, λ3`: Switching gains (higher = more robust, more chattering)

**Super-Twisting (STA) Parameters:**
- `k1, k2, k3`: Sliding surface gains
- `α1, α2, α3`: Super-twisting algorithm gains (continuous control)

**Adaptive SMC:**
- Automatically adjusts gains based on system uncertainty
- `γ` parameters control adaptation speed

### Performance Trade-offs:
- **Higher gains** → Faster response, more control effort
- **Lower gains** → Smoother control, slower response
- **Full dynamics** → More accurate but computationally intensive

### Experiment Ideas:
1. **Robustness Test**: Increase initial disturbance and compare controllers
2. **Efficiency Analysis**: Find minimum gains for stable control
3. **Comparative Study**: Test all controllers with identical conditions

---

## 🚀 Next Steps

This interactive notebook showcases our **professionally documented modules** in action!

**Explore more:**
- Use the Streamlit app for full animation and PSO optimization
- Check the `notebooks/` directory for specialized analysis notebooks
- Review our comprehensive documentation at `dip_docs/`

**For researchers:**
- Export simulation data for further analysis
- Customize controller implementations
- Integrate with your own optimization algorithms

*Built with ❤️ using our Task 2 enhanced documentation system*